In [ ]:
!pip install scikit-plot
!pip install jax-unirep

Try to deal with GPU memory 
accroading to https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html

In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt

from Bio import SeqIO
from jax_unirep import get_reps
from jax_unirep import evotune, fit
from jax_unirep.utils import dump_params
import os

os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='False'
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='.70'

/home/kongkitimanonk/.conda/envs/jupyter_NB/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
fastas = "/home/kongkitimanonk/SCRATCH_NOBAK/cd-hit/bk_fasta/ERR2130640.assembly.len15.fasta"
plk = "/home/kongkitimanonk/SCRATCH_NOBAK/phase3/Rat.len15.pkl"

In [3]:
with open(fastas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    seqs = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        # Remove leading and trailing characters from a string
        seqs.append(str(seq_record.seq.strip('*')))
        lengths.append(len(seq_record.seq))

In [4]:
# dictionary of lists  
dict = {'ID': identifiers, 'Sequence': seqs, 'length': lengths}  
df = pd.DataFrame(dict) 
#df["Sequence"] =  seqs
#df.to_pickle(plk)    
df  

,ID,Sequence,length
0,0,FFFLDDFYCRCTEGEEEGRGKDRV,24
1,1,EANYVHGPFLFPLPPPPCIYNKNHLKKKKKK,31
2,2,VRTRGILKGSWDLPFPI,17
3,3,TKEVTINADTTCGNDWVCEHRWRQIRNMVAFRNEV,35
4,4,LQKVKFIWYNNVINPTL,17
...,...,...,...
311702,311702,CGHGQGCSLPGPIWGWGG,18
311703,311703,KGFGWVQVSPPNENVVV,17
311704,311704,KSQEATDHWHITAAHSS,17
311705,311705,GWKSRGGWDHRTPRVPR,17


Remove Duplicate

In [5]:
df.drop_duplicates(subset=['Sequence'],inplace=True)
df

,ID,Sequence,length
0,0,FFFLDDFYCRCTEGEEEGRGKDRV,24
1,1,EANYVHGPFLFPLPPPPCIYNKNHLKKKKKK,31
2,2,VRTRGILKGSWDLPFPI,17
3,3,TKEVTINADTTCGNDWVCEHRWRQIRNMVAFRNEV,35
4,4,LQKVKFIWYNNVINPTL,17
...,...,...,...
311702,311702,CGHGQGCSLPGPIWGWGG,18
311703,311703,KGFGWVQVSPPNENVVV,17
311704,311704,KSQEATDHWHITAAHSS,17
311705,311705,GWKSRGGWDHRTPRVPR,17


In [6]:
_h_avg, h_final, c_final= get_reps(df['Sequence'].to_list())
df['reps']=_h_avg.tolist()
df

,ID,Sequence,length,reps
0,0,FFFLDDFYCRCTEGEEEGRGKDRV,24,"[0.02810720168054104, -0.002855583792552352, 0..."
1,1,EANYVHGPFLFPLPPPPCIYNKNHLKKKKKK,31,"[0.04690786451101303, -0.10334441810846329, -0..."
2,2,VRTRGILKGSWDLPFPI,17,"[0.03444882109761238, -0.04888810217380524, -0..."
3,3,TKEVTINADTTCGNDWVCEHRWRQIRNMVAFRNEV,35,"[0.01922222599387169, -0.07419043034315109, 0...."
4,4,LQKVKFIWYNNVINPTL,17,"[0.03477690368890762, -0.02958511933684349, -0..."
...,...,...,...,...
311702,311702,CGHGQGCSLPGPIWGWGG,18,"[0.031068434938788414, -0.09170962125062943, -..."
311703,311703,KGFGWVQVSPPNENVVV,17,"[0.04786856472492218, -0.01857958547770977, -0..."
311704,311704,KSQEATDHWHITAAHSS,17,"[0.03583816811442375, -0.058845095336437225, -..."
311705,311705,GWKSRGGWDHRTPRVPR,17,"[0.03462590277194977, -0.05176414176821709, -0..."


In [7]:
df.to_pickle(plk)
